<div style="text-align: center;">

<h2> <b>Curso:</b> Licenciatura em Ciência de Dados - 2º ano, 2º semestre</h2>

<h2> <b>Disciplina:</b> Otimização Heurística</h2>

<b><h3>MÉTODOS DE OTIMIZAÇÃO MULTIOBJECTIVO: <br>PROGRAMAÇÃO LINEAR POR METAS - NÃO PREEMPTIVA E PREEMPTIVA</h3></b>

<br>

<b>Docente</b>: Mafalda de Ponte

<b>Aluno</b>: José Ricardo de Almeida Valério - <b>Nº</b> 112255

<br>

<b>Repositório Github com este notebook e o relatório</b>: https://github.com/rvzzz/iscte-otimizacao-heuristica-projetos

</div>

---

#### Importação das bibliotecas Python necessárias para o trabalho:

In [43]:
from pulp import LpMinimize, LpProblem, LpStatus, LpVariable, GLPK
import numpy as np

### <span style="color:magenta;">Questão a) - Averiguar Viabilidade do Problema (considerando todas as restrições como hard)</span>

In [44]:
# criar o modelo de programação linear
model = LpProblem("Questao_a)", sense=LpMinimize)

# variáveis de decisão (não-negativas)
# x1 : nº de minutos de anúncios do Tipo I - Futebol
# x2 : nº de minutos de anúncios do Tipo II - Telenovelas
# x3 : nº de minutos de anúncios do Tipo III - Horário Nobre
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# restrições de audiências mínimas
model += (3 * x[1] + 1 * x[2] + 2 * x[3] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados
model += (5 * x[1] + 3 * x[2] + 4 * x[3] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total") # Custo total

# 6. Restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total") # Tempo total

# 3. Função objetivo fictícia (112255, pois só interessa testar a viabilidade)
obj_func = 112255
model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questao_a):
MINIMIZE
112255
SUBJECT TO
HRE: 3 x1 + x2 + 2 x3 >= 20

PRB: 5 x1 + 3 x2 + 4 x3 >= 30

MRE: 2.5 x1 + 2 x2 + 3 x3 >= 18

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [45]:
status = model.solve()

print(f"status: {LpStatus[status]} [{status}]\n")

if LpStatus[model.status] == "Optimal":
    print("É possível encontrar um plano de publicidade viável. \n\n")
else:
    print("Não é possível encontrar um plano de publicidade viável. \n\n")

status: Infeasible [-1]

Não é possível encontrar um plano de publicidade viável. 




### <span style="color:magenta;">Questão b) - Programação por Metas não Preemptiva</span>

#### b1) <span style="color:darkgoldenrod;">Soma Ponderada dos Desvios Percentuais</span> 

##### <span style="color:skyblue;">Plano 1:</span> $p_1 = p_2 = p_3 = 1$ 

In [46]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b1_plano1", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - o que falta para atingir a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(1, 4)}

# restrições de audiências mínimas
model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# valores meta e pesos para as audiências mínimas
import numpy as np
vam = np.array([20, 30, 18]) # valores meta
peso = np.array([1, 1, 1]) # pesos para as audiências mínimas

# função objetivo
obj_func = (peso[0]/vam[0]) * dm[1] + \
           (peso[1]/vam[1]) * dm[2] + \
           (peso[2]/vam[2]) * dm[3]

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b1_plano1:
MINIMIZE
0.05*dm1 + 0.03333333333333333*dm2 + 0.05555555555555555*dm3 + 0.0
SUBJECT TO
HRE: dm1 + 3 x1 + x2 + 2 x3 >= 20

PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
dm1 Continuous
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [47]:
status = model.solve(solver=GLPK(msg=True))

print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 0.22518513333333334

Solução Ótima:
   x1* = 3.06667
   x2* = 0.0
   x3* = 3.44444

Valores das variáveis de desvio:
   dm1* = 3.91111
   dm2* = 0.888889
   dm3* = 0.0

Valores Alcançados nas metas:
   20 - 3.91111 = 16.08889
   30 - 0.888889 = 29.111111
   18 - 0.0 = 18.0

Valores das restrições:
   HRE: -8.881784197001252e-16
   PRB: -9.999999999177334e-07
   MRE: -4.999999999810711e-06
   Custo_Total: 0.0
   Tempo_Min_Total: 1.51111


##### <span style="color:skyblue;">Plano 2:</span> $p_1 = 6$ e $p_2 = p_3 = 1$

In [48]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b1_plano2", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - o que falta para atingir a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(1, 4)}

# restrições de audiências mínimas
model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# valores meta e pesos para as audiências mínimas
vam = np.array([20, 30, 18]) # valores meta
peso = np.array([6, 1, 1]) # pesos para as audiências mínimas

# função objetivo
obj_func = (peso[0]/vam[0]) * dm[1] + \
           (peso[1]/vam[1]) * dm[2] + \
           (peso[2]/vam[2]) * dm[3]

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b1_plano2:
MINIMIZE
0.3*dm1 + 0.03333333333333333*dm2 + 0.05555555555555555*dm3 + 0.0
SUBJECT TO
HRE: dm1 + 3 x1 + x2 + 2 x3 >= 20

PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
dm1 Continuous
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [49]:
status = model.solve(solver=GLPK(msg=True))

print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 1.188611111111111

Solução Ótima:
   x1* = 5.65
   x2* = 0.0
   x3* = 0.0

Valores das variáveis de desvio:
   dm1* = 3.05
   dm2* = 1.75
   dm3* = 3.875

Valores Alcançados nas metas:
   20 - 3.05 = 16.95
   30 - 1.75 = 28.25
   18 - 3.875 = 14.125

Valores das restrições:
   HRE: 2.6645352591003757e-15
   PRB: 0.0
   MRE: 0.0
   Custo_Total: 0.0
   Tempo_Min_Total: 0.6500000000000004


#### b2) <span style="color:darkgoldenrod;">Minimização do desvio máximo de cada meta - Objetivo MiniMax</span> 

##### <span style="color:skyblue;">Plano 3:</span>   $p_1 = p_2 = p_3 = 1$

In [50]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b2_plano3", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - falta face à a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(1, 4)}

# variável MinMax - Q: é a variável que vai ser minimizada, ou seja, o valor máximo dos desvios de cada meta.
Q = LpVariable(name="Q", lowBound=0)

# restrições de audiências mínimas
model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# valores alvo das metas e pesos para as audiências mínimas
vam = np.array([20, 30, 18]) # valores alvo das meta ou níveis de aspiração
peso = np.array([1, 1, 1]) # pesos para as metas

# Adicionar as restrições de desvio: estas restrições de desvio são as que garantem que o valor máximo dos desvios de cada meta seja menor ou igual a Q.
model += (((peso[0]/vam[0]) * dm[1]) - Q <= 0, "desvio_inf_meta1")
model += (((peso[1]/vam[1]) * dm[2]) - Q <= 0, "desvio_inf_meta2")
model += (((peso[2]/vam[2]) * dm[3]) - Q <= 0, "desvio_inf_meta3")

# função objetivo
obj_func = Q

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b2_plano3:
MINIMIZE
1*Q + 0
SUBJECT TO
HRE: dm1 + 3 x1 + x2 + 2 x3 >= 20

PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

desvio_inf_meta1: - Q + 0.05 dm1 <= 0

desvio_inf_meta2: - Q + 0.0333333333333 dm2 <= 0

desvio_inf_meta3: - Q + 0.0555555555556 dm3 <= 0

VARIABLES
Q Continuous
dm1 Continuous
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [51]:
status = model.solve(solver=GLPK(msg=True))

print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 0.162963

Solução Ótima:
   x1* = 5.02222
   x2* = 0.0
   x3* = 0.837037

Valores das variáveis de desvio:
   dm1* = 3.25926
   dm2* = 1.54074
   dm3* = 2.93333

Valores Alcançados nas metas:
   20 - 3.25926 = 16.74074
   30 - 1.54074 = 28.45926
   18 - 2.93333 = 15.06667

Valores das restrições:
   HRE: -6.000000001282757e-06
   PRB: -1.199999999945689e-05
   MRE: -8.999999999481645e-06
   Custo_Total: -0.2249999999985448
   Tempo_Min_Total: 0.8592569999999999
   desvio_inf_meta1: 0.0
   desvio_inf_meta2: -0.111605
   desvio_inf_meta3: -2.222222222070247e-07


##### <span style="color:skyblue;">Plano 4:</span> $p_1 = p_2 = 1$ e $p_3 = 4$

In [52]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b2_plano4", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - falta face à a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(1, 4)}

# variável MinMax - Q: é a variável que vai ser minimizada, ou seja, o valor máximo dos desvios de cada meta.
Q = LpVariable(name="Q", lowBound=0)

# restrições de audiências mínimas
model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# valores alvo das metas e pesos para as audiências mínimas
vam = np.array([20, 30, 18]) # valores alvo das meta ou níveis de aspiração
peso = np.array([1, 1, 4]) # pesos para as metas

# Adicionar as restrições de desvio: estas restrições de desvio são as que garantem que o valor máximo dos desvios de cada meta seja menor ou igual a Q.
model += (((peso[0]/vam[0]) * dm[1]) - Q <= 0, "desvio_inf_meta1")
model += (((peso[1]/vam[1]) * dm[2]) - Q <= 0, "desvio_inf_meta2")
model += (((peso[2]/vam[2]) * dm[3]) - Q <= 0, "desvio_inf_meta3")

# função objetivo
obj_func = Q

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b2_plano4:
MINIMIZE
1*Q + 0
SUBJECT TO
HRE: dm1 + 3 x1 + x2 + 2 x3 >= 20

PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

desvio_inf_meta1: - Q + 0.05 dm1 <= 0

desvio_inf_meta2: - Q + 0.0333333333333 dm2 <= 0

desvio_inf_meta3: - Q + 0.222222222222 dm3 <= 0

VARIABLES
Q Continuous
dm1 Continuous
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [53]:
status = model.solve(solver=GLPK(msg=True))


print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 0.186243

Solução Ótima:
   x1* = 3.6254
   x2* = 0.0
   x3* = 2.69947

Valores das variáveis de desvio:
   dm1* = 3.72487
   dm2* = 1.07513
   dm3* = 0.838095

Valores Alcançados nas metas:
   20 - 3.72487 = 16.27513
   30 - 1.07513 = 28.92487
   18 - 0.838095 = 17.161905

Valores das restrições:
   HRE: 1.0000000000509601e-05
   PRB: 9.99999999806711e-06
   MRE: 4.999999998922533e-06
   Custo_Total: 0.25
   Tempo_Min_Total: 1.3248699999999998
   desvio_inf_meta1: 5.000000000143778e-07
   desvio_inf_meta2: -0.15040533333333334
   desvio_inf_meta3: 3.333333333521704e-07


### <span style="color:magenta;">Questão c) - Análise de Dominância das soluções encontradas em b)</span>

In [54]:
# dicionário com os planos de publicidade (1, 2, 3 e 4) e os respetivos valores em cada meta:
# valores absolutos que compõem a diferença entre os níveis de aspiração e os desvios encontrados
dif_entre_niveis_de_aspiracao_e_desvios_encontrados = {
    1: [16.09, 29.11, 18],
    2: [16.95, 28.25, 14.12],
    3: [16.74, 28.46, 15.07],
    4: [16.28, 28.92, 17.16]
}

# lista para armazenar os planos dominados que forem encontrados
planos_dominados = []

# verificar se algum plano domina outro
print("Verificação:")
for i in range(1, 5):
    for j in range(1, 5):
        if i != j:
            print(f"\tA verificar se plano {i} domina o plano {j}...")
            # verificar se o plano i domina o plano j
            if all(dif_entre_niveis_de_aspiracao_e_desvios_encontrados[i][k] <= dif_entre_niveis_de_aspiracao_e_desvios_encontrados[j][k] for k in range(3)):
                planos_dominados.append((i, j))
                print(f"\nPlano {i} domina o plano {j}.")
    print("\t------------------------------------------")


print("\n\nConclusão:")

# imprimir os planos dominados
if not planos_dominados:
    print("\tNão existem planos dominados.")
else:
    print("\nPlanos dominados encontrados:")
    for plano in planos_dominados:
        print(f"\tPlano {plano[0]} domina o plano {plano[1]}.")



Verificação:
	A verificar se plano 1 domina o plano 2...
	A verificar se plano 1 domina o plano 3...
	A verificar se plano 1 domina o plano 4...
	------------------------------------------
	A verificar se plano 2 domina o plano 1...
	A verificar se plano 2 domina o plano 3...
	A verificar se plano 2 domina o plano 4...
	------------------------------------------
	A verificar se plano 3 domina o plano 1...
	A verificar se plano 3 domina o plano 2...
	A verificar se plano 3 domina o plano 4...
	------------------------------------------
	A verificar se plano 4 domina o plano 1...
	A verificar se plano 4 domina o plano 2...
	A verificar se plano 4 domina o plano 3...
	------------------------------------------


Conclusão:
	Não existem planos dominados.


### <span style="color:magenta;">Questão d) - Programação por Metas Preemptiva</span>

#### Formulação do problema:

![Formulação do Problema de Programação Por Metas Preemptiva](https://raw.githubusercontent.com/rvzzz/iscte-otimizacao-heuristica-projetos/main/Trabalho%201/imagens/questao_d_formulacao_do_problema.png)

#### Formulação e Resolução do 1º nível:

![1º nível do Problema](https://raw.githubusercontent.com/rvzzz/iscte-otimizacao-heuristica-projetos/main/Trabalho%201/imagens/questao_d_1_nivel.png)

In [55]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b2_plano4", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - falta face à a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(2, 3)}

# restrições de audiências mínimas necessárias neste 1º nível:
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos

# para este 1º nível não iremos considerar as outras duas audiências:
# model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados
# model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# função objetivo
obj_func = dm[2]

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b2_plano4:
MINIMIZE
1*dm2 + 0
SUBJECT TO
PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
dm2 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [56]:
status = model.solve(solver=GLPK(msg=True))


print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 0.0

Solução Ótima:
   x1* = 0.0
   x2* = 0.0
   x3* = 7.5

Valores das variáveis de desvio:
   dm2* = 0.0

Valores Alcançados nas metas:
   20 - 0.0 = 20.0

Valores das restrições:
   PRB: 0.0
   Custo_Total: -2500.0
   Tempo_Min_Total: 2.5


#### Formulação e Resolução do 2º nível:

![2º nível do Problema](https://raw.githubusercontent.com/rvzzz/iscte-otimizacao-heuristica-projetos/main/Trabalho%201/imagens/questao_d_2_nivel.png)

In [57]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b2_plano4", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - falta face à a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(2, 4)}

# restrições de audiências mínimas necessárias neste 2º nível:
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados

# restrição com a informação encontrada no 1º nível:
model += (dm[2] == 0, "desvio_2")

# para este 2º nível não iremos considerar a outra:
# model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados

# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# função objetivo
obj_func = dm[3]

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b2_plano4:
MINIMIZE
1*dm3 + 0
SUBJECT TO
PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

desvio_2: dm2 = 0

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [58]:
status = model.solve(solver=GLPK(msg=True))


print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 0.0

Solução Ótima:
   x1* = 0.0
   x2* = 0.666667
   x3* = 7.0

Valores das variáveis de desvio:
   dm2* = 0.0
   dm3* = 0.0

Valores Alcançados nas metas:
   20 - 0.0 = 20.0
   30 - 0.0 = 30.0

Valores das restrições:
   PRB: 1.0000000010279564e-06
   MRE: 4.333334000000001
   desvio_2: 0.0
   Custo_Total: 0.02000000001862645
   Tempo_Min_Total: 2.6666670000000003


#### Formulação e Resolução do 3º nível:

![3º nível do Problema](https://raw.githubusercontent.com/rvzzz/iscte-otimizacao-heuristica-projetos/main/Trabalho%201/imagens/questao_d_3_nivel.png)

In [59]:
# criar o modelo de programação linear
model = LpProblem(name="Questão_b2_plano4", sense=LpMinimize)

# variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}

# veriáveis de desvio para as metas de audiências (desvios negativos - falta face à a meta)
dm = {i: LpVariable(name=f"dm{i}", lowBound=0) for i in range(1, 4)}

# restrições de audiências mínimas necessárias neste 2º nível:
model += (5 * x[1] + 3 * x[2] + 4 * x[3] + dm[2] >= 30, "PRB")   # PRB - Pessoas com Rendimentos Baixos
model += (2.5 * x[1] + 2 * x[2] + 3 * x[3] + dm[3] >= 18, "MRE") # MRE - Mulheres com Rendimentos Elevados
model += (3 * x[1] + 1 * x[2] + 2 * x[3] + dm[1] >= 20, "HRE")   # HRE - Homens com Rendimentos Elevados

# restrição com a informação encontrada no 1º nível:
model += (dm[2] == 0, "desvio_2")
model += (dm[3] == 0, "desvio_3")


# restrição de custo
model += (100_000 * x[1] + 60_000 * x[2] + 75_000 * x[3] <= 565_000, "Custo_Total")

# restrição de tempo total
model += (x[1] + x[2] + x[3] >= 5, "Tempo_Min_Total")

# função objetivo
obj_func = dm[1]

model += obj_func

# visualização do modelo matemático de forma a conferir os dados
model

Questão_b2_plano4:
MINIMIZE
1*dm1 + 0
SUBJECT TO
PRB: dm2 + 5 x1 + 3 x2 + 4 x3 >= 30

MRE: dm3 + 2.5 x1 + 2 x2 + 3 x3 >= 18

HRE: dm1 + 3 x1 + x2 + 2 x3 >= 20

desvio_2: dm2 = 0

desvio_3: dm3 = 0

Custo_Total: 100000 x1 + 60000 x2 + 75000 x3 <= 565000

Tempo_Min_Total: x1 + x2 + x3 >= 5

VARIABLES
dm1 Continuous
dm2 Continuous
dm3 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [60]:
status = model.solve(solver=GLPK(msg=True))


print("Valor ótimo:")
print(f"   Z* = {model.objective.value()}")

print("\nSolução Ótima:")
for var in x.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores das variáveis de desvio:")
for var in dm.values():
    print(f"   {var.name}* = {var.value()}")

print("\nValores Alcançados nas metas:")
for meta, desvio in zip(vam, dm.values()):
    print(f"   {meta} - {desvio.value()} = {meta - desvio.value()}")

print("\nValores das restrições:")
for name, constraint in model.constraints.items():
    print(f"   {name}: {constraint.value()}")

Valor ótimo:
   Z* = 4.8

Solução Ótima:
   x1* = 0.4
   x2* = 0.0
   x3* = 7.0

Valores das variáveis de desvio:
   dm1* = 4.8
   dm2* = 0.0
   dm3* = 0.0

Valores Alcançados nas metas:
   20 - 4.8 = 15.2
   30 - 0.0 = 30.0
   18 - 0.0 = 18.0

Valores das restrições:
   PRB: 0.0
   MRE: 4.0
   HRE: -8.881784197001252e-16
   desvio_2: 0.0
   desvio_3: 0.0
   Custo_Total: 0.0
   Tempo_Min_Total: 2.4000000000000004
